## Hidden Markov Model (HMM)

### Problem

- Input: vector $x:\{x_1,x_2,...,x_n\}\in O$
- Output: vector $y:\{y_1,y_2,...,y_d\}\in T$
- Where $x$ is the observation and $y$ is the hidden state
- $x,y$ is usually categorical, but can be numerical
- $x\in\mathbb R,y\in\mathbb R$

### Modelling

**Generative Tagging Model**  
$(x_1,...,x_n,\ y_1,...,y_n)\in S$ *(set of all observed pairs)*
$p(x_1,...,x_n,\ y_1,...,y_n)\geq0\forall(x_1,...,x_n,\ y_1,...,y_n)$
$\sum_{(x_1,...,x_n,\ y_1,...,y_n)\in S}p(x_1,...,x_n,\ y_1,...,y_n)=1$  
$\begin{aligned}y_1^*,...,y_n^*&=\arg\max_{y_1,...,y_n}(x_1,...,x_n,\ y_1,...,y_n)\\&=f(x_1,...,x_n)\end{aligned}$  

$\Pr(X_1=x_1,...,X_n=x_n,\ Y_1=y_1,...,Y_n=y_n)=p(x_1,...,x_n,\ y_1,...,y_n)$
- $Y_0=y_0=\text{"START"}$  
- $Y_{n+1}=y_{n+1}=\text{"STOP"}$  

**transition parameters**  
$p(x_1,...,x_n,\ y_0,y_1,...,y_{n+1})=p(y_0,...,y_{n+1})p(x_1,...,x_n\mid y_0,...,y_{n+1})$
$\begin{aligned}p(y_0,...,y_{n+1})&=p(y_0)p(y_1\mid y_0)p(y_2\mid y_0,y_1)...p(y_{n+1}\mid y_0,...,y_n)\\&\approx p(y_0)p(y_1\mid y_0)p(y_2\mid y_1)...p(y_{n+1}\mid y_n)\\&=p(y_1\mid y_0)...p(y_{n+1}\mid y_n)\\&=\prod_{i=1}^{n+1}p(y_i\mid y_{i-1})\end{aligned}$  
*(This is a markov sequence, independent given current tag)*  

**emission parameters**  
$\begin{aligned}p(x_1,...,x_n\mid y_0,...,y_{n+1})&=p(x_1\mid y_0,...,y_{n+1})p(x_2\mid x_1,\ y_0,...,y_{n+1})...p(x_n\mid x_1,...,x_{n-1},\ y_0,...,y_{n+1})\\&\approx p(x_1\mid y_1)p(x_2\mid y_2)...p(x_n\mid y_n)\\&=\prod_{i=1}^np(x_i\mid y_i)\end{aligned}$  
*(word only depends on tag)*  

**tagging model**  
$p(x_1,...,x_n,\ y_0,...,y_{n+1})=\prod_{i=1}^{n+1}p(y_i\mid y_{i-1})\prod_{i=1}^np(x_i\mid y_i)$  
**Generative Process**  
1. Set $y_0 = \text{START}$ and let $i=1$  
2. Generate tag $y_i$ from transition parameter $p(y_i\mid y_{i-1})$  
    - If $y_i=\text{STOP}$, terminate process and return $y_0,...,y_{i},x_1,...,x_{i-1}$
    - Otherwise, generate $x_i$ from emission parameter $p(x_i\mid y_i)$
3. Set $i=i+1$, repeat from Step 2  

**HMM Formal Definition**  
HMM is defined by $<T,O,\theta>$
- > $T$ is the set of states including $\text{START}$ and $\text{STOP}$
    - $\{0,1,...,\mid T\mid-1\}$ *(0 is START, |T|-1 is STOP)*
- > $O$ is the set of observed symbols *(i.e. x: words)*
- > $\theta=<a,b>$ are the parameters
    - transition parameter: $a_{u,v}=p(y_i=v\mid y_{i-1}=u)$
        - $\sum_{v'=1}^{\mid T\mid-1}a_{u,v'}=1 \forall u$
    - emission parameter: $b_u(o)=p(x=o\mid y=u)$
        - $\sum_{o\in O}b_u(o)=1 \forall u$


### Supervised Learning

- transition parameter: $a_{u,v}=\frac{\text{Count}(u;v)}{\text{Count}(u)}$
    - percentage of times v appears after u, given u
- emission parameter: $b_u(o)=\frac{\text{Count}(u\rightarrow o)}{\text{Count}(u)}$
    - percentage of times u emits symbol o

### Decoding

$\arg\max_{y_1,...,y_n}p(y_0,...,y_{n+1}\mid x_1,...,x_n;\theta)$  

$p(x_1,...,x_n,\ y_0,...,y_{n+1};\theta)=\prod_{i=1}^{n+1}a_{y_{i-1},y_i}\prod_{i=1}^nb_{y_i}(x_i)$  

$\begin{aligned}\arg\max_{y_1,...,y_n}p(y_0,...,y_{n+1}\mid x_1,...,x_n;\theta)&=\arg\max_{y_1,...,y_n}\frac{p(x_1,...,x_n,\ y_0,...,y_{n+1};\theta)}{p(x_1,...,x_n;\theta)}\\&=\arg\max_{y_1,...,y_n}p(x_1,...,x_n,\ y_0,...,y_{n+1};\theta)\end{aligned}$

### Vertibi Algorithm

joint probability of first k tags  
$r(y_1,...,y_k)=\prod_{i=1}^{k}a_{y_{i-1},y_i}\prod_{i=1}^kb_{y_i}(x_i)$  

$\begin{aligned}p(x_1,...,x_n,\ y_0,...,y_{n+1})&=r(y_1,...,y_n)\cdot a_{y_{n},y_{n+1}}\\&= r(y_1,...,y_n)\cdot a_{y_{n},\text{STOP}}\end{aligned}$

$\pi(k,v)=\max_{(y_1,...,y_k)\in S(k,v)}r(y_1,...,y_k)$
- $S(k,v)$ is the set of all sequences of length k with last tag v

**Base case**  
$\pi(0,u)=\left\{\begin{array}{cl}1 & \text{if }u=\text{START}\\0 & \text{otherwise}\end{array}\right.$  

**Forward recursively** $\forall k\in\{1,...,n\}$  
$\pi(k,v)\max_{u\in T}\{\pi(k-1,u)\cdot a_{u,v}\cdot b_v(x_k)\}$  

**Terminate** $y_n\text{ to STOP}$  
$\max_{y_1,...,y_n}p(x_1,...,x_n,\ y_0,...,y_{n+1})=\max_{v\in T}\{\pi(n,v)\cdot a_{v,\text{STOP}}\}$  

**Backtracking**  
$y_n^*=\arg\max_v\{\pi(n,v)\cdot a_{v,\text{STOP}}\}$  
$y_{n-1}^*=\arg\max_u\{\pi(n-1,u)\cdot a_{u,y_n^*}\}$  
*repeat for all y*

### Unsupervised Learning

#### Hard EM

**E-Step**  
observation sequence: $\boldsymbol x^{(i)}= (x_1^{(i)},x_2^{(i)},...,x_n^{(i)})$  
state sequence *(viterbi)*: $\boldsymbol y^{(i)*}= (y_1^{(i)*},y_2^{(i)*},...,y_n^{(i)*})$  

$\text{Count}(u;v), \text{Count}(u)\ \forall u,v\in T=\{0,1,...,N+1\}$  
$\text{Count}(u\rightarrow o), \text{Count}(u)\ \forall u\in T,o\in O$  

**M-Step**  
$a_{u,v}=\frac{\text{Count}(u;v)}{\text{Count}(u)}$  
$b_u(o)=\frac{\text{Count}(u\rightarrow o}{\text{Count}(u)}$  


#### Soft EM


**E-Step**  
$\text{Count}(u;v)=\sum_{i=1}^m\sum_yp(\boldsymbol y\mid\boldsymbol x^{(i)})\text{Count}(\boldsymbol x^{(i)},\boldsymbol y,u\rightarrow v)$
- where $\text{Count}(\boldsymbol x^{(i)},\boldsymbol y,u\rightarrow v)$ is number of times a trasition from u to v occurs in tag sequence y corresponding to observation sequence $x^{(i)}$  
 
**M-Step**  
$a_{u,v}=\frac{\text{Count}(u;v)}{\sum_{v'}\text{Count}(u;v')}$
- where denominator ensures $\sum_{v'=1}^{N+1}a_{u,v'}=1$  

marginal posterior probabilities: $p(y_j=u,y_{j+1}=v\mid\boldsymbol x;\theta)=\sum_{\boldsymbol y:y_j=u,y_{j+1}=v}p(\boldsymbol y\mid\boldsymbol x;\theta)$  
$\sum_yp(\boldsymbol y\mid\boldsymbol x^{(i)})\text{Count}(\boldsymbol x^{(i)},\boldsymbol y,u\rightarrow v)=\sum_{j=0}^np(y_j=u,y_{j+1}=v\mid\boldsymbol x^{(i)};\theta)$  
$P()$


### Forward-Backward Algorithm

For every state sequence y there is
- a path through the graph $\text{START}, <1,y_1>,...,<n,y_n>,\text{STOP}$
- state sequence $\boldsymbol y=y_1,...,y_n$ has score equal to $p(\boldsymbol x,\boldsymbol y;\theta)$
- sum of scores for all paths from START to state (j,u), $\alpha_u(j)$
- sum of scores for all paths from state (j,u) to STOP, $\beta_u(j)$  
Given $x_1,...,x_n\forall u\in 1,...,N,\ j\in1,...,n$  

**forward probability**  
$\alpha_u(j)=p(x_1,...,x_{j-1},y_j=u;\theta)$
- base case: $\alpha_u(1)=a_{\text{START},u}\forall u\in1,...,N-1$
- recursive case: $\alpha_u(j+1)=\sum_v\alpha_v(j)a_{v,u}b_v(x_j)\forall u\in1,...,N-1,\ j\in1,...,n-1$  

**backward probability**  
$\beta_u(j)=p(x_j,...,x_n\mid y_j=u;\theta)$
- base case: $\beta_u(n)=a_{u,\text{STOP}}b_u(x_n)\forall u\in1,...,N-1$
- recursive case: $\beta_u(j)=\sum_va_{u,v}b_u(x_j)\beta_u(j+1)\forall u\in1,...,N-1,\ j\in n-1,...,1$  

**Transition parameters**  
$\text{Count}(u;v)=\sum_{i=1}^m\text{Count}^{(i)}(u;v)$  
expected number of times we see u,v pair from ith instance  
$\text{Count}^{(i)}(u;v)=\sum_{j=0}^n\frac{1}{Z}\alpha_u(j)b_u(k_j^{(i)})\alpha_{u,v}\beta_u(j+1)$  
$Z=\sum_{u'}\alpha_{u'}(j)\beta_{u'}(j)$  

$\text{Count}^{(i)}(u)=\sum_{j=1}^n\frac{1}{Z}\alpha_u(j)\beta_u(j)$  
$Z=\sum_{u'}\alpha_{u'}(j)\beta_{u'}(j) \forall j\in1,...,n$  

**emission parameters**  
$\text{Count}(u\rightarrow o)=\sum_{i=1}^m\text{Count}^{(i)}(u\rightarrow o)$  
$\text{Count}^{(i)}(u\rightarrow)=\sum_{j:x_j^{(i)}=o}\frac{1}{Z}\alpha_u(j)\beta_u(j)$  
$Z=\sum_{u'}\alpha_{u'}(j)\beta_{u'}(j)$  
    
**max marginal decoding**  
$\begin{aligned}y_i^*&=\arg\max_up(y_i=u\mid x_1,...,x_n;\theta)\\&=\arg\max_u\frac{\alpha_u(i)\beta_u(i)}{\sum_v\alpha_v(j)\beta_v(j)}\\&=\arg\max_u\alpha_u(i)\beta_u(i)\end{aligned}$



# Reinforcement Learning

![model](D:/Documents/projects/201709.university.t6/assets/201712150748.PNG)

## Value Iteration algorithm

![](D:/Documents/projects/201709.university.t6/assets/201712150751.PNG)

## Q-Value Iteration Algorithm

![](D:/Documents/projects/201709.university.t6/assets/201712150752.PNG)

## Q Learning Algorithm

![](D:/Documents/projects/201709.university.t6/assets/201712150754.PNG)